In [121]:
import numpy as np
import json
from geo_boundaries import *
from county_geo import *
from sklearn.externals import joblib
from sklearn.neighbors import KDTree
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import cnfg
import json
from geo import *
# from vaderSentiment.vaderSentiment import sentiment as vaderSentiment
# from textblob import TextBlob
from pymongo import MongoClient
import time
import logging
import logging.handlers
import datetime
import os.path
from bson.json_util import dumps
from __future__ import division
import math

In [122]:
# client = MongoClient()
# tweets = client.fletcher.tweets
# backup = tweets.find({"$text":{"$search":"\"hillary clinton\", hillary, clinton"}})
# hillary = tweets.aggregate([{"$match":{"$text":{"$search":"hillary"}}}])
# ,
#      {"$match":{"timestamp_ms":{"$gte":100000}}}])
# all_tweets = tweets.find()

##### New Bootstrap Function based on Machine Learning

In [154]:
client = MongoClient()
tweets = client.fletcher.tweets
hillary_terms = "hillaryclinton hillary teamhillary teamclinton teamhillaryclinton hillary2016 clinton2016 hillaryclinton2016 hillaryforamerica hillary4america hillaryforpresident hillary4president"
hillary = tweets.aggregate([{"$match":{"$text":{"$search":hillary_terms}}}])

In [155]:
# put tweets in document
import re
from collections import defaultdict
from gensim.models.ldamodel import LdaModel
documents = []
limit = 0
pattern = re.compile("[^a-zA-Z ]")
for tweet in hillary:
    if limit < 50:
        documents.append(pattern.sub('', tweet['text']))
        limit += 1

##### Twitter Stop Words

In [156]:
stopwords = ["@", "a", "about", "above", "across", "after", "again", "against", "all", "almost", "alone", "along", "already", "also", "although", "always", "among", "an", "and", "another", "any", "anybody", "anyone", "anything", "anywhere", "are", "area", "areas", "around", "as", "ask", "asked", "asking", "asks", "at", "away", "b", "back", "backed", "backing", "backs", "be", "became", "because", "become", "becomes", "been", "before", "began", "behind", "being", "beings", "best", "better", "between", "big", "both", "but", "by", "c", "came", "can", "cannot", "case", "cases", "certain", "certainly", "clear", "clearly", "come", "could", "d", "did", "differ", "different", "differently", "do", "does", "done", "down", "down", "downed", "downing", "downs", "during", "e", "each", "early", "either", "end", "ended", "ending", "ends", "enough", "even", "evenly", "ever", "every", "everybody", "everyone", "everything", "everywhere", "f", "face", "faces", "fact", "facts", "far", "felt", "few", "find", "finds", "first", "for", "four", "from", "full", "fully", "further", "furthered", "furthering", "furthers", "g", "gave", "general", "generally", "get", "gets", "give", "given", "gives", "go", "going", "good", "goods", "got", "great", "greater", "greatest", "group", "grouped", "grouping", "groups", "h", "had", "has", "have", "having", "he", "her", "here", "herself", "high", "high", "high", "higher", "highest", "him", "himself", "his", "how", "however", "i", "if", "important", "in", "interest", "interested", "interesting", "interests", "into", "is", "it", "its", "itself", "j", "just", "k", "keep", "keeps", "kind", "knew", "know", "known", "knows", "l", "large", "largely", "last", "later", "latest", "least", "less", "let", "lets", "like", "likely", "long", "longer", "longest", "m", "made", "make", "making", "man", "many", "may", "me", "member", "members", "men", "might", "more", "most", "mostly", "mr", "mrs", "much", "must", "my", "myself", "n", "necessary", "need", "needed", "needing", "needs", "never", "new", "new", "newer", "newest", "next", "no", "nobody", "non", "noone", "not", "nothing", "now", "nowhere", "number", "numbers", "o", "of", "off", "often", "old", "older", "oldest", "on", "once", "one", "only", "open", "opened", "opening", "opens", "or", "order", "ordered", "ordering", "orders", "other", "others", "our", "out", "over", "p", "part", "parted", "parting", "parts", "per", "perhaps", "place", "places", "point", "pointed", "pointing", "points", "possible", "present", "presented", "presenting", "presents", "problem", "problems", "put", "puts", "q", "quite", "r", "rather", "really", "right", "right", "room", "rooms", "s", "said", "same", "saw", "say", "says", "second", "seconds", "see", "seem", "seemed", "seeming", "seems", "sees", "several", "shall", "she", "should", "show", "showed", "showing", "shows", "side", "sides", "since", "small", "smaller", "smallest", "so", "some", "somebody", "someone", "something", "somewhere", "state", "states", "still", "still", "such", "sure", "t", "take", "taken", "than", "that", "the", "their", "them", "then", "there", "therefore", "these", "they", "thing", "things", "think", "thinks", "this", "those", "though", "thought", "thoughts", "three", "through", "thus", "to", "today", "together", "too", "took", "toward", "turn", "turned", "turning", "turns", "two", "u", "under", "until", "up", "upon", "us", "use", "used", "uses", "v", "very", "w", "want", "wanted", "wanting", "wants", "was", "way", "ways", "we", "well", "wells", "went", "were", "what", "when", "where", "whether", "which", "while", "who", "whole", "whose", "why", "will", "with", "within", "without", "work", "worked", "working", "works", "would", "x", "y", "year", "years", "yet", "you", "young", "younger", "youngest", "your", "yours", "z"]
candidate_search = { "Lincoln Chafee" : "lincolnchafee chafee teamchafee teamlincolnchafee chafee2016 lincolnchafee2016 chafeeforpresident chafee4president",  "Hillary Clinton" : "hillaryclinton hillary teamhillary teamclinton teamhillaryclinton hillary2016 clinton2016 hillaryclinton2016 hillaryforamerica hillary4america hillaryforpresident hillary4president",  "Martin O'Malley" : "governoromalley martinomalley teamomalley teammartinomalley omalley2016 martinomalley2016 omalley4president omalleyforpresident",  "Bernie Sanders" : "berniesanders sensanders teambernie teamberniesanders bernie2016 sanders2016 berniesanders2016 sanders4president sandersforpresident bernie4president bernieforpresident berniesanderse4president berniesanderseforpresident",  "Ben Carson" : "realbencarson bencarson drbencarson teamcarson teambencarson bencarson2016 carson2016 carson4president carsonforpresident bencarson4president bencarsonforpresident",  "Ted Cruz" : "tedcruz sentedcruz teamcruz teamtedcruz tedcruz2016 cruzcrew cruz4president cruzforpresident tedcruz4president tedcruzforpresident",  "Carly Fiorina" : "carlyfiorina fiorina teamcarly teamfiorina teamcarlyfiorina carly2016 carly4president carlyforpresident carlyfiorina4president carlyfiorinaforpresident fiorina4president fiorinaforpresident",  "Lindsey Graham" : "linseygrahamsc lindseygraham grahamblog senlindseygraham teamgraham teamlindseygraham graham2016 lindseygraham2016 graham4president grahamforpresident lindseygraham4president lindseygrahamforpresident",  "George Pataki" : "governorpataki govpataki pataki georgepataki pataki4president patakiforpresident georgepataki4president georgepatakiforpresident teampataki",  "Rand Paul" : "randpaul drrandpaul senatorrandpaul teamrand teamrandpaul randpaul2016 rand2016 standwithrand rand4president randforpresident paul4president paulforpresident randpaul4president randpaulforpresident",  "Rick Perry" : "governorperry rickperry teamrickperry perry2016 rickperry2016 perry4president perryforpresident rickperry4president rickperryforpresident",  "Marco Rubio" : "marcorubio rubio2016 marcorubio2016 teammarco teammarcorubio marco4president marcoforpresident rubio4president rubioforpresident marcorubio4president marcorubioforpresident",  "Rick Santorum" : "ricksantorum santorum2016 teamsantorum teamricksantorum santorum4president santorumforpresident ricksantorum4president ricksantorumforpresident",  "Mike Huckabee" : "govmikehuckabee mikehuckabee teamhuckabee teammikehuckabee huckabee2016 mikehuckabee2016 huckaboom huckabee4president huckabeeforpresident mikehuckabee4president mikehuckabeeforpresident",  "Jeb Bush" : "jebbush  teamjeb teamjebbush jeb2016 bush2016 jebbush2016 jeb jeb4president jebforpresident jebbush4president jebbushforpresident",  "Scott Walker" : "scottwalker teamscottwalker scottwalker2016 walker4president walkerforpresident scottwalker4president scottwalkerforpresident", "Donald Trump" : "donaldtrump realdonaldtrump teamdonaldtrump teamtrump trump2016 donaldtrump2016 donaltrump4president donaldtrumpforpresident trump4president trumpforpresident"}
candidate_stop_words = {}
# # def add_twitter_stop_words():
for key in candidate_search:
    candidate_stop_words[key] = []
    for word in candidate_search[key].split():
        candidate_stop_words[key].append(word)

In [157]:
# candidate_stop_words

In [158]:
hillary_stop = candidate_stop_words['Hillary Clinton'] + stopwords

In [159]:
# remove common words and tokenize

stoplist = set(hillary_stop)
texts = [[word for word in document.lower().split() if word not in stoplist]
        for document in documents]
# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
        for text in texts]

from pprint import pprint   # pretty-printer
# pprint(texts)

In [160]:
dictionary = corpora.Dictionary(texts)
# dictionary.save('hillary.dict')
# for item in dictionary:
#     print item
print dictionary.items()

[(10, u'freedomspark'), (32, u'square'), (7, u'dangora'), (23, u'topic'), (17, u'saturday'), (5, u'row'), (31, u'memorial'), (24, u'trending'), (27, u'ny'), (3, u'freedoms'), (34, u'amazing'), (9, u'tom'), (15, u'nyc'), (12, u'fellowsnv'), (33, u'southpoint'), (18, u'trndnl'), (38, u'watch'), (35, u'beechman'), (4, u'park'), (22, u'hours'), (25, u'arrived'), (29, u'york'), (28, u'party'), (6, u'line'), (20, u'hillarysupervols'), (14, u'stage'), (37, u'theater'), (26, u'look'), (13, u'hillaryfornv'), (16, u'rights'), (1, u'island'), (2, u'roosevelt'), (30, u'fdr'), (36, u'laurie'), (0, u'franklin'), (21, u'time'), (39, u'knocked'), (11, u'view'), (8, u'thehrcsupervols'), (19, u'history')]


In [161]:
# print(dictionary.token2id)

In [162]:
corpus = [dictionary.doc2bow(text) for text in texts]
# corpora.MmCorpus.serialize('hillary.mm', corpus)
# print(corpus)

In [163]:
# class MyCorpus(object):
#     def __iter__(self):
#         for line in open("corpustxt"):
#             yield dictionary.doc2bow(line.lower().split())

In [164]:
from gensim.models.ldamodel import LdaModel

In [142]:
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=1, update_every=1, chunksize=10000, passes=10)
return lda.print_topics(1)

In [143]:
# lda.print_topics(1)

[u'0.097*roosevelt + 0.092*park + 0.065*franklin + 0.065*freedoms + 0.032*saturday + 0.032*island + 0.027*trndnl + 0.027*tom + 0.022*dangora + 0.022*thehrcsupervols']

In [173]:
def get_topic_dictionary(candidate):
    client = MongoClient()
    tweets = client.fletcher.tweets
    tweets = tweets.aggregate([{"$match":{"$text":{"$search":candidate_search[candidate]}}}])
    documents = []
    pattern = re.compile("[^a-zA-Z ]")
    for tweet in tweets:
        documents.append(pattern.sub('', tweet['text']))
    stoplist = set(candidate_stop_words[candidate] + stopwords)
    texts = [[word for word in document.lower().split() if word not in stoplist]
            for document in documents]
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    texts = [[token for token in text if frequency[token] > 1]
            for text in texts]
    dictionary = corpora.Dictionary(texts)
    topics = []
    for item in dictionary.items():
        topics.append({"text" : item[1], "size" : item[0]})
    return topics

In [174]:
get_topic_dictionary("Hillary Clinton")

[{'size': 578, 'text': u'liar'},
 {'size': 268, 'text': u'forget'},
 {'size': 341, 'text': u'grassroots'},
 {'size': 960, 'text': u'skip'},
 {'size': 178, 'text': u'rob'},
 {'size': 622, 'text': u'focus'},
 {'size': 239, 'text': u'thehill'},
 {'size': 190, 'text': u'steve'},
 {'size': 871, 'text': u'breitbartnews'},
 {'size': 121, 'text': u'aides'},
 {'size': 783, 'text': u'talks'},
 {'size': 419, 'text': u'follow'},
 {'size': 635, 'text': u'werent'},
 {'size': 160, 'text': u'primer'},
 {'size': 756, 'text': u'hate'},
 {'size': 560, 'text': u'children'},
 {'size': 252, 'text': u'issues'},
 {'size': 13, 'text': u'row'},
 {'size': 470, 'text': u'rd'},
 {'size': 188, 'text': u'voter'},
 {'size': 688, 'text': u'standar'},
 {'size': 300, 'text': u'itll'},
 {'size': 693, 'text': u'standa'},
 {'size': 605, 'text': u'send'},
 {'size': 524, 'text': u'champions'},
 {'size': 704, 'text': u'proudwarriordad'},
 {'size': 754, 'text': u'list'},
 {'size': 1019, 'text': u'bike'},
 {'size': 1018, 'text'

In [132]:
volume = {}
sentiment = {}
    
for tweet in all_tweets:
    try:
        if volume[tweet['fips']] > 0:        
            try:
                volume[tweet['fips']] += 1
                try:
                    sentiment[tweet['fips']] += tweet['sentiment']['compound']
                except:
                    pass
            except:
                pass
        else:
            volume[tweet['fips']] = 1
            sentiment[tweet['fips']] = tweet['sentiment']['compound']
    except:
        pass

for key in sentiment:
    sentiment[key] /= volume[key]

In [134]:
volume

{}

##### Gensim Code

In [2]:
from gensim import corpora, models, similarities
client = MongoClient()
tweets = client.fletcher.tweets


NameError: name 'MongoClient' is not defined

In [ ]:
# client = MongoClient()
# tweets = client.fletcher.tweets

#### Test Code

In [120]:
# all_tweets = tweets.find()
# backup.count()
# query = tweets.aggregate([
#             {"$match":{"$text":{"$search":"hillary"}}}])

In [170]:
# count = 0
# for tweet in query:
#     count += 1

# print count
from __future__ import division
import flask
from pymongo import MongoClient
import time
from county_geo import *
from twitter_functions import *
import math
from datetime import date

def return_tweets(day, search_terms="", all_results=0):
    today_epoch = int(date.today().strftime('%s'))
    start_time = (today_epoch - (day*86400))*1000
    end_time = ((start_time + 86400)*1000)-1
    if all_results==1:
        start_time = 0
        end_time = (today_epoch+86400)*1000
    client = MongoClient()
    db = client.fletcher.tweets
    tweets = {}
    if search_terms != "":
        query = db.aggregate([
            {"$match":{"$text":{"$search":search_terms}}},
            {"$match":{"timestamp_ms":{"$gte":start_time,"$lt":end_time}}}])
    else:
        query = db.aggregate([
            {"$match":{"timestamp_ms":{"$gte":start_time,"$lt":end_time}}}])
    for tweet in query:
        fips = tweet['fips']
        if fips in tweets:
            tweets[fips]['volume'] += 1
            try:
                tweets[fips]['sentiment'] += tweet['sentiment']['compound']
            except:
                pass
        else:
            tweets[fips] = {}
            tweets[fips]['volume'] = 1
            try:
                tweets[fips]['sentiment'] = tweet['sentiment']['compound']
            except:
                tweets[fips]['sentiment'] = 0
    for x in tweets:
        tweets[x]['sentiment'] = tweets[x]['sentiment']/tweets[x]['volume']

    return tweets

def tweet_booststrapper(dict, n=0):
    bootstrap_dict = {}
    for key in all_fips:
        try:
            volume = dict[key]['volume']
            sentiment = dict[key]['sentiment']
        except:
            volume = 0
            sentiment = None
        if volume < 100:
            new_sent, new_vol, weight, new_mult = 0, 0, 10, 0
            for county in nearest_counties[key]:
                try:
                    vol = dict[county]['volume']*weight
                    new_vol += vol
                    new_sent += dict[county]['sentiment']*multiplier
                    new_mult += multiplier
                except:
                    pass
                weight -= 1
            new_vol = (volume*(volume/100))+((new_vol/55)*((100-volume)/100))
            if new_mult > 0:
                new_sent = (volume*(volume/100))+((new_sent/new_mult)*((100-volume)/100))
            else:
                new_sent = sentiment
        else:
            new_vol = volume
            new_sent = sentiment
        bootstrap_dict[key] = {}
        bootstrap_dict[key]['volume'] = new_vol
        bootstrap_dict[key]['sentiment'] = new_sent
    if n > 0:
        n -= 1
        return tweet_booststrapper(bootstrap_dict, n)
    return bootstrap_dict


def get_all_candidates(time=0, n=0, all_results=0):
    candidate_search = { "Lincoln Chafee" : "lincolnchafee chafee teamchafee teamlincolnchafee chafee2016 lincolnchafee2016 chafeeforpresident chafee4president",  "Hillary Clinton" : "hillaryclinton hillary teamhillary teamclinton teamhillaryclinton hillary2016 clinton2016 hillaryclinton2016 hillaryforamerica hillary4america hillaryforpresident hillary4president",  "Martin O'Malley" : "governoromalley martinomalley teamomalley teammartinomalley omalley2016 martinomalley2016 omalley4president omalleyforpresident",  "Bernie Sanders" : "berniesanders sensanders teambernie teamberniesanders bernie2016 sanders2016 berniesanders2016 sanders4president sandersforpresident bernie4president bernieforpresident berniesanderse4president berniesanderseforpresident",  "Ben Carson" : "realbencarson bencarson drbencarson teamcarson teambencarson bencarson2016 carson2016 carson4president carsonforpresident bencarson4president bencarsonforpresident",  "Ted Cruz" : "tedcruz sentedcruz teamcruz teamtedcruz tedcruz2016 cruzcrew cruz4president cruzforpresident tedcruz4president tedcruzforpresident",  "Carly Fiorina" : "carlyfiorina fiorina teamcarly teamfiorina teamcarlyfiorina carly2016 carly4president carlyforpresident carlyfiorina4president carlyfiorinaforpresident fiorina4president fiorinaforpresident",  "Lindsey Graham" : "linseygrahamsc lindseygraham grahamblog senlindseygraham teamgraham teamlindseygraham graham2016 lindseygraham2016 graham4president grahamforpresident lindseygraham4president lindseygrahamforpresident",  "George Pataki" : "governorpataki govpataki pataki georgepataki pataki4president patakiforpresident georgepataki4president georgepatakiforpresident teampataki",  "Rand Paul" : "randpaul drrandpaul senatorrandpaul teamrand teamrandpaul randpaul2016 rand2016 standwithrand rand4president randforpresident paul4president paulforpresident randpaul4president randpaulforpresident",  "Rick Perry" : "governorperry rickperry teamrickperry perry2016 rickperry2016 perry4president perryforpresident rickperry4president rickperryforpresident",  "Marco Rubio" : "marcorubio rubio2016 marcorubio2016 teammarco teammarcorubio marco4president marcoforpresident rubio4president rubioforpresident marcorubio4president marcorubioforpresident",  "Rick Santorum" : "ricksantorum santorum2016 teamsantorum teamricksantorum santorum4president santorumforpresident ricksantorum4president ricksantorumforpresident",  "Mike Huckabee" : "govmikehuckabee mikehuckabee teamhuckabee teammikehuckabee huckabee2016 mikehuckabee2016 huckaboom huckabee4president huckabeeforpresident mikehuckabee4president mikehuckabeeforpresident",  "Jeb Bush" : "jebbush  teamjeb teamjebbush jeb2016 bush2016 jebbush2016 jeb jeb4president jebforpresident jebbush4president jebbushforpresident",  "Scott Walker" : "scottwalker teamscottwalker scottwalker2016 walker4president walkerforpresident scottwalker4president scottwalkerforpresident" }
    candidate_tweets = {}
    candidate_totals = {}
    for key in candidate_search:
        total_volume = 0
        total_sentiment = 0
        total_sent_vol = 0
        search_terms = candidate_search[key]
        tweets = return_tweets(time, search_terms, all_results)
        boosted_tweets = tweet_booststrapper(tweets,n)
        candidate_tweets[key] = boosted_tweets
        for county in boosted_tweets:
            county_volume = boosted_tweets[county]['volume']
            county_sentiment = boosted_tweets[county]['sentiment']
            total_volume += county_volume
            if county_sentiment != None:
                total_sentiment += (county_volume*county_sentiment)
                total_sent_vol += county_volume
        candidate_totals[key] = {}
        try:
            candidate_totals[key]['volume'] = total_volume
            candidate_totals[key]['sentiment'] = total_sentiment/total_sent_vol
        except:
            candidate_totals[key]['volume'] = 0
            candidate_totals[key]['sentiment'] = 0
    all_data = {"total" : candidate_totals, "county" : candidate_tweets}
    return all_data

def get_all_candidates_daily(time=1):
    day_array = {}
    for x in range(time):
        day_array[x] = get_all_candidates(x)
    return day_array

def get_candidates_js_object(time=0, n=0):
    candidates_object = []
    js_date = date.today() - timedelta(time)
    js_date = js_date.strftime('%Y-%m-%d')
    for key in candidate_search:
        total_volume = 0
        total_sent_vol = 0
        search_terms = candidate_search[key]
        tweets = return_tweets(time, search_terms)
        boosted_tweets = tweet_booststrapper(tweets,n)
        candidate_dict = { "value" : key, "date" : js_date }
        for county in boosted_tweets:
            county_volume = boosted_tweets[county]['volume']
            total_volume += county_volume
        try:
            candidate_dict['volume'] = total_volume
        except:
            candidate_dict['volume'] = 0
        candidates_object.append(candidate_dict)
    return candidates_object

def get_all_candidates_js_objects(time=1):
    all_candidates_object = []
    for x in range(time):
        all_candidates_object += get_candidates_js_object(x)
    return all_candidates_object

In [171]:
# all_candidates = get_all_candidates(1000)
# tweets = tweet_booststrapper(return_tweets(1))
# tweets = return_tweets(1)
test = get_all_candidates_js_objects(10)

In [172]:
# tweets
# all_candidates
test

[{'date': '2015-06-16', 'value': 'Rand Paul', 'volume': 0.0},
 {'date': '2015-06-16', 'value': 'Rick Santorum', 'volume': 0.0},
 {'date': '2015-06-16', 'value': 'Scott Walker', 'volume': 0.0},
 {'date': '2015-06-16', 'value': "Martin O'Malley", 'volume': 0.0},
 {'date': '2015-06-16', 'value': 'Lindsey Graham', 'volume': 0.0},
 {'date': '2015-06-16', 'value': 'Jeb Bush', 'volume': 0.0},
 {'date': '2015-06-16', 'value': 'Hillary Clinton', 'volume': 0.0},
 {'date': '2015-06-16', 'value': 'Rick Perry', 'volume': 0.0},
 {'date': '2015-06-16', 'value': 'Donald Trump', 'volume': 0.0},
 {'date': '2015-06-16', 'value': 'George Pataki', 'volume': 0.0},
 {'date': '2015-06-16', 'value': 'Marco Rubio', 'volume': 0.0},
 {'date': '2015-06-16', 'value': 'Mike Huckabee', 'volume': 0.0},
 {'date': '2015-06-16', 'value': 'Bernie Sanders', 'volume': 0.0},
 {'date': '2015-06-16', 'value': 'Ted Cruz', 'volume': 0.0},
 {'date': '2015-06-16', 'value': 'Carly Fiorina', 'volume': 0.0},
 {'date': '2015-06-16', '

In [168]:
# all_candidates
bs_tweets = tweet_booststrapper(tweets)

In [169]:
bs_tweets

{54065: {'sentiment': 0.08868292682926829, 'volume': 105.80909090909091},
 54011: {'sentiment': 0.0992646351242983, 'volume': 242.92727272727274},
 42009: {'sentiment': 0.08742803347280335, 'volume': 81.27272727272728},
 46101: {'sentiment': 0.0796736842105263, 'volume': 42.236363636363635},
 30007: {'sentiment': 0.0, 'volume': 36.627272727272725},
 30043: {'sentiment': 0.022288235294117647, 'volume': 28.87272727272727},
 9003: {'sentiment': 0.14354445747800584, 'volume': 1922.3545454545456},
 41003: {'sentiment': 0.13749286498353455, 'volume': 361.72727272727275},
 41005: {'sentiment': 0.16612479414455616, 'volume': 1600.1909090909091},
 41007: {'sentiment': 0.10159348795718111, 'volume': 317.3818181818182},
 41009: {'sentiment': 0.08471231527093599, 'volume': 705.6818181818181},
 41011: {'sentiment': 0.0997190082644628, 'volume': 165.39090909090908},
 41013: {'sentiment': 0.13577419354838707, 'volume': 52.972727272727276},
 41015: {'sentiment': 0.16174875000000002, 'volume': 103.0909

In [11]:
fips = {}

for num in all_fips:
    fips[num] = [0,0]

for doc in all_tweets:
    fips_num = doc['fips']
    try:
        fips_sent = doc['sentiment']['compound']
    except:
        fips_sent = 0
    if fips_num in fips:
        fips[fips_num][0] += 1
        fips[fips_num][1] += fips_sent
        
for key in fips:
    if fips[key][0] != 0:
        fips[key][1] = float(fips[key][1]/fips[key][0])

sentiment = {}
tweets_total = {}
tweet_density = {}

for key in fips:
    sentiment[key] = fips[key][1]
    tweets_total[key] = fips[key][0]
    tweet_density[key] = (fips[key][0]/fips_pop[key])*1000

In [20]:
no_tweets = []
for k, v in tweets_total.iteritems():
    if v == 0:
        no_tweets.append(k)

In [22]:
# tweets_total

In [36]:
# nearest_counties_dictionary = {}

# for key in geo_county_dictionary:
#     nearest_counties = find_county(geo_county_dictionary[key], neighbors=11, all=1)
#     nearest_counties_dictionary[key] = nearest_counties[1:]
    
# json.dump(nearest_counties_dictionary, open("nearest_counties_dictionary.py",'w'))

def county_booster(fips_dict, other_dict):
    boosted_dict = {}
    for key in fips_dict:
        total_vol = fips_dict[key]
        new_score = total_vol*other_dict[key]*15
        total = (15*total_vol)
        if total_vol < 10:
            index = 10
            for county in nearest_counties[key]:
                new_score += (other_dict[county]*index)
                index -= 1
        new_score = new_score / (total+55)
        boosted_dict[key] = new_score
    return boosted_dict

In [31]:
# county_booster(tweets_total, sentiment)

0.110476612392


{41045: 0.11047661239168362}

In [37]:
boosted_sentiment = county_booster(tweets_total, sentiment)
boosted_density = county_booster(tweets_total, tweet_density)

In [38]:
with open('boosted_sentiment.tsv', 'w') as f:
    [f.write('{0}\t{1}\n'.format(key, value)) for key, value in boosted_sentiment.items()]

In [39]:
with open('tweets.tsv', 'w') as f:
    [f.write('{0}\t{1}\n'.format(key, value)) for key, value in tweets_total.items()]

In [40]:
with open('boosted_tweet_density.tsv', 'w') as f:
    [f.write('{0}\t{1}\n'.format(key, value)) for key, value in boosted_density.items()]

In [62]:
# new_dict = {}
# for key in all_county_boundaries:
#     points = all_county_boundaries[key]
#     nest_level = find_nested(points)
#     if nest_level == 0:
#         new_array = []
#         for point in points:
#             new_array.append([point[1],point[0]])
#         new_dict[key] = new_array
#     else:
#         new_array = []
#         for sub_points in points:
#             sub_array = []
#             for point in sub_points:
#                 sub_array.append([point[1],point[0]])
#             new_array.append(sub_array)
#         new_dict[key] = new_array

In [67]:
# f = open('new_file.py', 'w')
# f.write(str(new_dict)) 
# f.close()

In [7]:
# county_geo_dictionary[40.776557, -73.970174]

In [96]:
def in_polygon(point, poly):
  x = point[0]
  y = point[1]
  n = len(poly)
  status = False
  p1x,p1y = poly[0]
  for i in range(n+1):
    p2x,p2y = poly[i % n]
    if min(p1y,p2y) < y <= max(p1y,p2y):
      if x <= max(p1x,p2x):
        if p1y != p2y:
          xinters = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
        if p1x == p2x or x <= xinters:
          status = not status
    p1x,p1y = p2x,p2y

  return status

def find_nested(nested_list):
    try:
        nested_list[0][0][0]
        nest = 1
    except:
        nest = 0
    return nest
    
def find_county(point):
    county_tree = joblib.load('geo_tree/county_tree.pkl') 
    dist, indices = county_tree.query(point, k=5)
    fips_list = []
    for index in indices[0]:
        try:
            fips = county_geo_dictionary[(county_geo[index][0],county_geo[index][1])]
            fips_list.append(fips)
        except:
            pass
    if len(fips_list) > 0:
        try:
            for fips_num in fips_list:
                try:
                    county_paths = all_county_boundaries[str(fips_num)]
                    if find_nested(county_paths) == 0:
                        if in_polygon(point, county_paths):
                            return fips_num
                            break
                    else:
                        for path in county_paths:
                            try:
                                if in_polygon(point, path):
                                    return fips_num
                                    break
                            except:
                                pass
                except:
                    pass
        except:
            return fips_list[0]
    else:
        return None
    if dist[0][0] > 6:
        return None
    return fips_list[0]

In [97]:
print find_county([40.533548, -97.290977])

31151


In [14]:
locations = [["bronx", [[40.915132, -73.912168],[40.799059, -73.918005],[40.814391, -73.779302],[40.881655, -73.782735]]], ["harlem", [[40.834370, -73.950320],[40.818913, -73.961478],[40.801243, -73.959247],[40.797084, -73.949290],[40.817614, -73.934012],[40.828006, -73.934871]]], ["upper_west", [[40.818913, -73.961478],[40.801243, -73.959247],[40.768767, -73.982015],[40.773057, -73.993516]]], ["upper_east", [[40.797084, -73.949290],[40.817614, -73.934012],[40.764275, -73.972852],[40.758798, -73.958775],[40.776219, -73.941094],[40.799159, -73.928306],]], ["midtown", [[40.801243, -73.959247],[40.768767, -73.982015],[40.764275, -73.972852],[40.758798, -73.958775],[40.726881, -73.971736],[40.745416, -74.015509]]], ["downtown", [[40.726881, -73.971736],[40.745416, -74.015509],[40.697279, -74.021947],[40.711984, -73.971307]]], ["brooklyn", [[40.738697, -73.965254],[40.707995, -73.972120],[40.704351, -73.996839],[40.674674, -74.029112],[40.607982, -74.049024],[40.559485, -74.020185],[40.601727, -73.820371],[40.694981, -73.869810]]], ["queens", [[40.694981, -73.869810],[40.738697, -73.965254],[40.791007, -73.908958],[40.801923, -73.775748],[40.731196, -73.705710],[40.574394, -73.754462],[0.544137, -73.952216],[40.566049, -73.915824],[40.601727, -73.820371]]], ["san_fran", [[37.707196, -122.503175],[37.711270, -122.348679],[37.837188, -122.364366],[37.800303, -122.527787]]], ["west_bay", [[37.416526, -121.994411],[37.361970, -122.120067],[37.627261, -122.498996],[37.707196, -122.503175],[37.711270, -122.348679]]], ["east_bay", [[37.486844, -121.872875],[37.451966, -121.947033],[37.502835, -122.102708],[37.961171, -122.443285],[38.080176, -122.165880]]], ["south_bay", [[37.486844, -121.872875],[37.451966, -121.947033],[37.416526, -121.994411],[37.361970, -122.120067],[37.222672, -121.927807],[37.305189, -121.750652]]]]
def find_location(point):
  for area in locations:
    if in_polygon(point,area[1]):
      return area[0]
      break
  else:
    return None

print find_location([40.749898, -73.985726])

midtown
